# Question-Answer Generator



Auto generate question answering pairs for paragraphs from documents. Evaluate LLM based on synthetic QA pairs.

Based on https://github.com/langchain-ai/auto-evaluator/tree/main licensed under Elastic License 2.0 (ELv2)

In [2]:
!python --version

Python 3.10.8


In [3]:
!pip install --upgrade sagemaker --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.132 requires botocore==1.29.132, but you have botocore 1.31.21 which is incompatible.
awscli 1.27.132 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0.1 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import sagemaker
import boto3
import botocore

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::843197046435:role/service-role/AmazonSageMaker-ExecutionRole-20230626T095793
sagemaker bucket: sagemaker-eu-west-1-843197046435
sagemaker session region: eu-west-1


## Load documents

In [5]:
s3 = sess.boto_session.resource("s3")
bucket_name = "gen-ai-foundation"
s3_key = "crawlers/admin-ch/admin_ch_press_releases.json"
output_bucket = "sagemaker-eu-west-1-843197046435"
s3_output_dataset_key = "admin_ch_dataset/train"
obj1 = s3.Object(bucket_name, s3_key)
jsonfile_content_str = obj1.get()["Body"].read().decode("utf-8")

In [6]:
import json

question_answers = json.loads(jsonfile_content_str)

In [7]:
import pandas as pd

df_origin = pd.DataFrame.from_dict(question_answers)

In [8]:
df_origin.iloc[:1]

,title,byline,dir,lang,content,textContent,length,excerpt,siteName,source,target_urls,link_text
0,Press releases from the Federal Administration,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\t\t\t\n\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t11.0...,7660,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",root


In [9]:
df = df_origin

In [10]:
dataset_key_to_generate_qa_for = "paragraphs"
dataset_key_to_generate_qa_for = "textContent"

## Use Falcon

In [15]:
import boto3
import json

# Create a low-level client representing Amazon SageMaker Runtime
# sagemaker_runtime = boto3.client("sagemaker-runtime", region_name="eu-west-1")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = "falcon-40b-instruct-48xl-5"

# After you deploy a model into production using SageMaker hosting
# services, your client applications use this API to get inferences
# from the model hosted at the specified endpoint.
# response = sagemaker_runtime.invoke_endpoint(
#                             EndpointName=endpoint_name,
#                             Body=bytes('{"features": ["This is great!"]}', 'utf-8') # Replace with your own data.
#                             )

# Optional - Print the response body and decode it so it is human read-able.
# print(response['Body'].read().decode('utf-8'))


client = boto3.client("sagemaker-runtime")
request = {
    "inputs": "The first paragraph of the book 'Alice in Wonderland' by Lewis Carroll reads as:",
    "parameters": {"truncation": True},
}

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(request),
)
print(response["Body"].read().decode())

[{"generated_text":"The first paragraph of the book 'Alice in Wonderland' by Lewis Carroll reads as:\n'Alice was beginning to get very tired of sitting by her sister on the bank, and of"}]


## Create Prompt to generate question answer pairs

In [61]:
number_of_questions_to_generate = 5

In [70]:
# falcon_prompt = """
# >>INTRODUCTION<<
# {}

# Generate a list of """+ str(number_of_questions_to_generate) +""" question and answer pairs for the text.

# When coming up with this question/answer pair, you must respond in the following format:
# ```
# [\n"""+(" {{\n  \"question\": \"...\",\n  \"answer\": \"...\"\n }},\n"*number_of_questions_to_generate)[:-2]+"""\n]
# ```
# Everything between the ``` must be valid json.


# Assistant: [
# """

In [82]:
# falcon_prompt

In [83]:
# prompt_response_format = """
# The JSON result should be inside <format></format> XML tag.
# <format>
# [
#  {{
#   \"question\": \"...\",
#   \"answer\": \"...\"
#  }},
#  {{
#   \"question\": \"...\",
#   \"answer\": \"...\"
#  }},
#  ...
# ]

# </format>

# Text:
# {}


# Assistant: Here is a list of """ + str(number_of_questions_to_generate) +""" question and answer pairs extracted from the text: ["""

# prompts = [

#     """Human: Generate a list of """ + str(number_of_questions_to_generate) +""" question and answer pairs for the following text.""" + prompt_response_format,
#     """Human: You are a smart assistant designed to help high school teachers come up with reading comprehension questions.
# Given a piece of text, you must come up with a question and answer pair that can be used to test a student's reading comprehension abilities.
# Generate a list of """ + str(number_of_questions_to_generate) +""" question and answer pairs for the following text.""" + prompt_response_format
# ]

In [84]:
falcon_kwargs = {
    "parameters": {
        "do_sample": True,
        "top_p": 0.95,
        "temperature": 0.1,
        "top_k": 50,
        "max_new_tokens": 1000,
        "repetition_penalty": 1.03,
        "stop": ["<|end|>", "<|endoftext|>", "]"],
    }
}

In [85]:
import json


def run_llm(text):
    # prompt_template = prompts[0]
    falcon_prompt = (
        f"""
    >>INTRODUCTION<<
    {text}

    Generate a list of """
        + str(number_of_questions_to_generate)
        + """ question and answer pairs for the text.

    When coming up with this question/answer pair, you must respond in the following format:
    ```
    [\n"""
        + (
            ' {{\n  "question": "...",\n  "answer": "..."\n }},\n'
            * number_of_questions_to_generate
        )[:-2]
        + """\n]
    ```
    Everything between the ``` must be valid json.



    Assistant: [
    """
    )

    prompt_template = falcon_prompt

    # parameters = {
    #     "max_tokens_to_sample": 600,
    # # "temperature": 1,
    # # "top_k": 250,
    # # "top_p": 0.999,
    # "stop_sequences": ["\\n\\nHuman:", "</format>","<format>", "<paragraphs>","[","]"],
    #     }
    prompt_data = prompt_template.format(text)
    prompt_data = falcon_prompt
    # print("------START-PROMPT-DATA---------------------")
    # # print(prompt_data)
    body = json.dumps({"inputs": prompt_data, **falcon_kwargs})
    # print("------END-PROMPT-DATA---------------------")
    # print(body)
    # print("------END-BODY---------------------")
    # modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
    accept = "application/json"
    contentType = "application/json"

    # response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    # response_body = json.loads(response.get('body').read())
    # return "["+response_body.get('completion')+"]"
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=contentType,
        Accept=accept,
        # Body=json.dumps(request),
        Body=body,
    )
    return response["Body"].read().decode()


res = run_llm(df[dataset_key_to_generate_qa_for][4])
print(json.loads(res))

[{'generated_text': '\n    >>INTRODUCTION<<\n    \n\n    Federal Statistical Office\n                \n                    Neuchâtel,\xa013.07.2023\xa0-\xa0In 2022, the 11 major border regions outside Switzerland recorded lower activity rates than the Swiss regions bordering them, but higher rates than their respective national rates. Unemployment rates based on ILO definition were lower than the national averages in the countries concerned and in some cases lower than those of their Swiss neighbours. This illustrates the positive impact of the Swiss labour market on these regions, in particular the employment of 373 000 foreign cross-border commuters. These are some of the results from the publication “L’offre de travail en Suisse et dans les régions limitrophes en 2022”, available in French or German, from the Federal Statistical Office (FSO).\n                \n            \n\n            \n            \n                This press release and further information on this topic can be

In [89]:
print(json.loads(res)[0]["generated_text"])


    >>INTRODUCTION<<
    

    Federal Statistical Office
                
                    Neuchâtel, 13.07.2023 - In 2022, the 11 major border regions outside Switzerland recorded lower activity rates than the Swiss regions bordering them, but higher rates than their respective national rates. Unemployment rates based on ILO definition were lower than the national averages in the countries concerned and in some cases lower than those of their Swiss neighbours. This illustrates the positive impact of the Swiss labour market on these regions, in particular the employment of 373 000 foreign cross-border commuters. These are some of the results from the publication “L’offre de travail en Suisse et dans les régions limitrophes en 2022”, available in French or German, from the Federal Statistical Office (FSO).
                
            

            
            
                This press release and further information on this topic can be found on the FSO website (see link below)

In [90]:
# initial the column where the list of qa pairs has to be stored
df["qa_pairs"] = None

In [ ]:
import time
from json import JSONDecodeError
import tqdm
import json

for i in tqdm.tqdm(range(df.shape[0])):
    current_res = df["qa_pairs"].values[i]
    if current_res is None or len(current_res) == 0:
        row = df.iloc[i]
        paragraph = row[dataset_key_to_generate_qa_for]

        try:
            qa_pairs = run_llm(paragraph)
            list_pairs = json.loads(qa_pairs)
            df["qa_pairs"].values[i] = list_pairs
            # time.sleep(12)
            time.sleep(1)
        except Exception as e:
            # print("------------------------")
            # print(qa_pairs)
            print("------------------------")
            print(e)
            print("------------------------")
            print("Sleeping")
            # time.sleep(12)
            time.sleep(1)
            df["qa_pairs"].values[i] = []
    else:
        print(f"Skipping row: {str(i)}")
        continue


print(len(df["qa_pairs"]))

  0%|          | 0/210 [00:00<?, ?it/s]

------------------------
An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "{"error":"Input validation error: `inputs` tokens + `max_new_tokens` must be <= 2048. Given: 1849 `inputs` tokens and 1000 `max_new_tokens`","error_type":"validation"}". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/falcon-40b-instruct-48xl-5 in account 843197046435 for more information.
------------------------
Sleeping


  0%|          | 1/210 [00:01<03:45,  1.08s/it]

------------------------
An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "{"error":"Input validation error: `inputs` tokens + `max_new_tokens` must be <= 2048. Given: 2138 `inputs` tokens and 1000 `max_new_tokens`","error_type":"validation"}". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/falcon-40b-instruct-48xl-5 in account 843197046435 for more information.
------------------------
Sleeping


  2%|▏         | 5/210 [00:42<36:07, 10.58s/it]

...

------------------------
An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "{"error":"Input validation error: `inputs` tokens + `max_new_tokens` must be <= 2048. Given: 1268 `inputs` tokens and 1000 `max_new_tokens`","error_type":"validation"}". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/falcon-40b-instruct-48xl-5 in account 843197046435 for more information.
------------------------
Sleeping


 97%|█████████▋| 203/210 [33:47<00:45,  6.53s/it]

------------------------
An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "{"error":"Input validation error: `inputs` tokens + `max_new_tokens` must be <= 2048. Given: 1747 `inputs` tokens and 1000 `max_new_tokens`","error_type":"validation"}". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/falcon-40b-instruct-48xl-5 in account 843197046435 for more information.
------------------------
Sleeping


100%|██████████| 210/210 [35:07<00:00, 10.04s/it]

210


In [93]:
# show rows for which qa generation did not succeed
df[df["qa_pairs"].apply(lambda x: True if x is None or len(x) == 0 else False)]

,title,byline,dir,lang,content,textContent,length,excerpt,siteName,source,target_urls,link_text,qa_pairs
0,Press releases from the Federal Administration,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\t\t\t\n\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t11.0...,7660,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",root,[]
1,Press releases from the Federal Administration,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\n\t\t\t\t...,9030,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",root,[]
5,co-operate: model for climate-friendly constru...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Laboratory for Materials Testi...,5907,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tco-operate: model for climate-...,[]
15,"On 25th anniversary of Rome Statute, Switzerla...",The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n \n\n\n\n\t\n\n\t\n\n\t\n\t\t\n\n\t\n\t...,5504,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tOn 25th anniversary of Rome St...,[]
16,Earlier detection of breast cancer,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Paul Scherrer Institut\n ...,8514,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tEarlier detection of breast ca...,[]
18,New EU policy measures based on Empa research:...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Laboratory for Materials Testi...,6580,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tNew EU policy measures based o...,[]
22,Quantum technology: Masters of defects,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Laboratory for Materials Testi...,8762,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tQuantum technology: Masters of...,[]
35,First report on cantonal measures relating to ...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Department of Finance\n ...,5268,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tFirst report on cantonal measu...,[]
36,Until the chemistry is just right,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Laboratory for Materials Testi...,9186,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tUntil the chemistry is just right,[]
38,"First recovery, then stagnation: The state of ...",The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Eawag: Swiss Federal Institute of Aqua...,7649,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...","\n\t\t\t\t\t\t\tFirst recovery, then stagnatio...",[]


In [108]:
df["qa_pairs"]

KeyError: 'qa_pairs'

In [96]:
df.iloc[:4]

,title,byline,dir,lang,content,textContent,length,excerpt,siteName,source,target_urls,link_text,qa_pairs
0,Press releases from the Federal Administration,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\t\t\t\n\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t11.0...,7660,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",root,[]
1,Press releases from the Federal Administration,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\n\t\t\t\t...,9030,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",root,[]
2,State Secretary Hirayama attends informal EU m...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...","\n\n State Secretariat for Education, Resea...",3230,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tState Secretary Hirayama atten...,[{'generated_text': ' >>INTRODUCTION<< ...
3,Issue result for Confederation bonds,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Finance Administration\n ...,1602,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tIssue result for Confederation...,[{'generated_text': ' >>INTRODUCTION<< ...


In [97]:
df_clean = df.explode("qa_pairs")
df_clean = df_clean[df_clean["qa_pairs"].notna()]
df_clean.reset_index(drop=True, inplace=True)
df_clean = df_clean.join(pd.json_normalize(df_clean.qa_pairs))
df_clean.drop("qa_pairs", axis=1, inplace=True)

In [116]:
df_clean.iloc[:1]

0    \n    >>INTRODUCTION<<\n    \n\n    State Secr...
Name: generated_text, dtype: object


In [99]:
df = df_clean

## Prepare input

In [100]:
import numpy as np

In [104]:
intro = pd.Series(
    np.full(df.shape[0], fill_value="The conversation between human and AI assistant.")
)
human_tag = pd.Series(np.full(df.shape[0], fill_value="\n[|Human|] "))
ai_tag = pd.Series(np.full(df.shape[0], fill_value="\n[|AI|] "))

In [106]:
df.head(5)

,title,byline,dir,lang,content,textContent,length,excerpt,siteName,source,target_urls,link_text,generated_text
0,State Secretary Hirayama attends informal EU m...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...","\n\n State Secretariat for Education, Resea...",3230,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tState Secretary Hirayama atten...,\n >>INTRODUCTION<<\n \n\n State Secr...
1,Issue result for Confederation bonds,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Finance Administration\n ...,1602,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tIssue result for Confederation...,\n >>INTRODUCTION<<\n \n\n Federal Fi...
2,Switzerland's border regions: higher activity ...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Statistical Office\n ...,1723,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tSwitzerland's border regions: ...,\n >>INTRODUCTION<<\n \n\n Federal St...
3,Producer and Import Price Index remained stabl...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Statistical Office\n ...,1499,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tProducer and Import Price Inde...,\n >>INTRODUCTION<<\n \n\n Federal St...
4,Extension of investigation activities to Xplai...,The Federal Council,None,en,"<div id=""readability-page-1"" class=""page""><div...",\n\n Federal Data Protection and Informatio...,1529,Current information from the Federal Administr...,None,https://www.admin.ch/gov/en/start/documentatio...,"[{'text': 'Homepage', 'url': 'https://www.admi...",\n\t\t\t\t\t\t\tExtension of investigation act...,\n >>INTRODUCTION<<\n \n\n Federal Da...


In [102]:
df["input"] = intro + human_tag + df["question"] + ai_tag + df["answer"] + human_tag

KeyError: 'question'

In [ ]:
df["input"][1]

In [103]:
len(df)

158

In [ ]:
df = df.drop_duplicates(subset=["input"])

In [ ]:
len(df)

## Store dataset

!pip install datasets[s3]

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

dataset = dataset.train_test_split(test_size=0.1, seed=2303)
dataset

In [ ]:
from datasets.filesystems import S3FileSystem

session = botocore.session.get_session()
s3 = S3FileSystem(session=session)

In [ ]:
dataset.save_to_disk(
    f"s3://{output_bucket}/{s3_output_dataset_key}", storage_options=s3.storage_options
)